In [40]:
import time
import os
import pandas as pd

from langchain_openai import AzureChatOpenAI
from datetime import datetime

In [2]:
data = pd.read_csv('Length Based Evaluation.csv')
data.head()

,question,zephyr3b_shard,llama2_7b,llama2_shard,tiny_llama,rocket_3b,marx_3b,stablelm_3b,zephyr3b_shard_score,llama2_7b_score,llama2_shard_score,tiny_llama_score,rocket_3b_score,marx_3b_score,stablelm_3b_score
0,What are the main services offered by banks?,answer: Banks offer a wide range of financial...,Banks offer various services to their customer...,Banks offer a wide range of services to thei...,"Banks provide a variety of services, including...","As an AI language model, I do not offer any sp...",Banks offer a wide array of financial products...,The main services offered by banks include che...,1.929224,4.767884,4.950000,0.076104,1.442161,0.228311,0.050000
1,How do banks make money?,"bank's profits, and to make investments that p...",Banks make money by charging interest on loans...,Banks make money through a variety of activi...,"Banks lend out money to people in need, or if ...",BiIt depends on\t\r\nSome banks generate incom...,"Banks generate revenue through various means, ...",Banks make money through various sources such ...,2.966621,0.950272,4.950000,0.050000,3.351499,0.435967,1.597411
2,What is the difference between a checking acco...,Host: Interesting question! A checking account...,A checking account and a savings account are b...,A checking account and a savings account are...,A checking account is a type of primary accoun...,A checking and a savings account are two types...,A checking account typically allows customers ...,A checking account is designed for routine tra...,2.374444,4.950000,4.876033,0.565798,2.126510,0.050000,1.026701
3,What is a credit union?,"In a nutshell, a credit union is a not-for-pro...",A credit union is a not-for-profit financial c...,A credit union is a financial cooperative th...,A credit union is a cooperative owned and cont...,"Well, it doesn't sound like you're really from...",A credit union is an association of people who...,A credit union is a cooperative organization t...,0.584939,3.359221,1.235745,4.950000,1.358632,0.054070,0.050000
4,What is a debit card?,A debit card is a plastic card linked to <|sy...,A debit card is a type of card that allows you...,A debit card is a type of payment card that ...,A debit card is a plastic credit card with a b...,"Well, I would say that it's not just an adject...",A debit card is an alternative to carrying cas...,A debit card is a type of bank card that allow...,0.050000,0.590523,4.950000,0.219236,1.676096,0.668317,0.576379


In [3]:
data.columns

Index(['question', 'zephyr3b_shard', 'llama2_7b', 'llama2_shard', 'tiny_llama',
       'rocket_3b', 'marx_3b', 'stablelm_3b', 'zephyr3b_shard_score',
       'llama2_7b_score', 'llama2_shard_score', 'tiny_llama_score',
       'rocket_3b_score', 'marx_3b_score', 'stablelm_3b_score'],
      dtype='object')

In [4]:
process = []
for col in data.columns:
    if 'score' not in col:
        process.append(col)

In [5]:
process

['question',
 'zephyr3b_shard',
 'llama2_7b',
 'llama2_shard',
 'tiny_llama',
 'rocket_3b',
 'marx_3b',
 'stablelm_3b']

In [28]:
model = AzureChatOpenAI(
    api_key='7a089cda15374dc2b5c25a62946c86db',
    azure_endpoint='https://dalle-azure.openai.azure.com/',
    model='gpt-4',
    azure_deployment='gpt-4-free',
    temperature=0.2,
    api_version='2023-07-01-preview'
)

In [29]:
prompt = f""""
Read the following question and answer and score in based on following paramters-
[
    'answer_correctness': #rate out of 10,
    'answer_length': #rate out of 10,
    'answer_formatting': #rate out of 5,
    'context_understanding': #rate out of 5,
]

Question {data['question'][5]}
Answer
{data['zephyr3b_shard'][5]}

please give your output in the form of a python dict as shown above. 
No need to explain or type anything else.

"""

In [30]:
answer = model.invoke(prompt)

In [31]:
answer.content

"{'answer_correctness': 10, 'answer_length': 2, 'answer_formatting': 1, 'context_understanding': 5}"

In [51]:
for col in process[2:]:
    tmp = []

    for i in range(len(data['question'])):
        
        start = time.time()

        prompt = f""""
Read the following question and answer and score in based on following paramters-
[
    'answer_correctness': #rate out of 10 can use float value,
    'answer_length': #rate out of 10 can use float value,
    'answer_formatting': #rate out of 5 can use float value,
    'context_understanding': #rate out of 5 can use float value,
]

Question - {data['question'][i]}
Answer - {data[col][i]}

Please give your output in the form of a python dict as shown above. 

Judging criteria
Answer should be correct with reference to the question
Answer length is expected to be at least 1 paragraph (150-300 words)
Answer should be well formated with proper punctuations ans should be in English only.
If any answer scores less than 5 on correctness metric, all other metric scores shouls also be less than 50% of their max available score.
If any answer scores 0 on correctness, all other criteria should be rated 0 only.

Refrain from giving full marks in metric. 
If an answer contains weird charaters or not written properly, feel free to deduct marks as well.
No need to explain or type anythiing else.

"""
        
        response = model.invoke(prompt)

        if len(response.content) > 80: 
            tmp.append(response.content)
            print(f'{col} {i} \n{response.content} \r', end='')

        else:
            tmp.append(None)
            print(f'Error - {response.content}')

        end = time.time()

        if end-start < 60/6:
            time.sleep((60/6) - (end-start))
            

    out_col = col + '_eval'
    data[out_col] = tmp

zephyr3b_shard 0 
{
    'answer_correctness': 9.5,
    'answer_length': 9.0,
    'answer_formatting': 4.5,
    'context_understanding': 5.0
zephyr3b_shard 1 
{
    'answer_correctness': 9.5,
    'answer_length': 9.0,
    'answer_formatting': 4.5,
    'context_understanding': 5.0
zephyr3b_shard 2 
{
    'answer_correctness': 9.5,
    'answer_length': 10,
    'answer_formatting': 4.5,
    'context_understanding': 5
zephyr3b_shard 3 
{
    'answer_correctness': 9.5,
    'answer_length': 10,
    'answer_formatting': 4.5,
    'context_understanding': 5
zephyr3b_shard 4 
{
    'answer_correctness': 0,
    'answer_length': 0,
    'answer_formatting': 0,
    'context_understanding': 0
zephyr3b_shard 5 
{
    'answer_correctness': 7.0,
    'answer_length': 2.0,
    'answer_formatting': 4.0,
    'context_understanding': 4.0
zephyr3b_shard 6 
{
    'answer_correctness': 7.5,
    'answer_length': 8,
    'answer_formatting': 2,
    'context_understanding': 4
zephyr3b_shard 7 
{
    'answer_correctn

KeyboardInterrupt: 